<a href="https://colab.research.google.com/github/padiketeku/Earth-Observation-Data-Programming/blob/main/Activity_2_Exploring_EE_in_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity 2- Exploring earth engine in COLAB

In [55]:
# import required pandas and geopandas
import pandas as pd
import geopandas as gpd

# import earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

## Autheticate Earth Engine

1.   You need your Earth Engine project ID, which you can obtain from Assets
2. The project ID is the argument for the ee.Initialize()



In [21]:
# Trigger the authentication command.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-racrabbe3')

## Set up objects to filter the image collection

In [22]:
# coordinates of the Camp Fire
lat =  39.444012
lon = -121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2018-10-01'

# end date
end_date = '2019-01-31'

## Collect the Landsat image from Earth Engine and filter this

In [23]:
# get the landsat 8 image collection
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

This is an image collection, which means that more than one image file is available to work with. You may want to know the exact number of images in the collect. To do this, use **size** method. The **getInfo()** methods would alse be used to retrieve the computed value from the server.

In [26]:
# print the number of images in the collection
print('Total number:', landsat.size().getInfo())

Total number: 8


The collection has 8 images.

Also, it is ideal to know the spectral bands making up the image. Once you see the bands google search "Landsat 8 bands" so you can explain each bands, noting relevant and less relevant bands for a typical remote sensing project.

In [31]:
# find the spectral bands
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

Another information you may be curious about is the acquisition date. For instance, you would like to know the acquisition date for the first image in the collection.

In [27]:
# acquisition date for the first image in the collection
landsat.first().get('DATE_ACQUIRED').getInfo()

'2018-10-07'

The first image was acquired on the 7th October 2018

## Cloud cover in multispectral optical imagery

> Add blockquote



We did not filter the collection by cloud cover. Since cloud cover is a major issue in optical remote sensing, you may want to have an idea on the amount of cloud cover in the image. This helps you to gauge the quality of the image, and hence decide whether to use it straight away, discard it or mask the pixels affected by cloud cover before application.

In [30]:
# check the cloud cover
landsat.first().get('CLOUD_COVER_LAND').getInfo()

0.05

### Visualisation of the Landsat imagery

You now know there are 8 images in the collection and the first image has 0.05% cloud cover. You may be wondering if the other images have similar, lower or higher percentage cloud cover. And you might even be more keen to know whether the cloud cover is affecting your study area. This is the time to visualise each image of the collection and assess cloud cover for each acquisition date.  

In [34]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

In [35]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [37]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER_LAND').getInfo()

    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)

    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data
    data.append(this_data)




Image # 0 2018-10-07 Cloud cover: 0.05


Image # 1 2018-10-23 Cloud cover: 73.03


Image # 2 2018-11-08 Cloud cover: 11.84


Image # 3 2018-11-24 Cloud cover: 67.17


Image # 4 2018-12-10 Cloud cover: 56.08


Image # 5 2018-12-26 Cloud cover: 5.99


Image # 6 2019-01-11 Cloud cover: 80.06


Image # 7 2019-01-27 Cloud cover: 5.19


**Question- which image (acquisition date) had the most cloud cover?**

## Make an attribute table

Aside from the images, you may want to create an attribute table for easy understanding of the data. We would create a table that shows the acquisition date and cloud cover for each image in the collection.

In [41]:
# To make a table the pandas DataFrame is used
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

# display the data frame
df

,Image #,Date,Cloud Cover
0,0,2018-10-07,0.05
1,1,2018-10-23,73.03
2,2,2018-11-08,11.84
3,3,2018-11-24,67.17
4,4,2018-12-10,56.08
5,5,2018-12-26,5.99
6,6,2019-01-11,80.06
7,7,2019-01-27,5.19


## Select and zoom images

Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [42]:
# create a list of images we want (before, during, after)
landsat_sequence = [0,2,5]

In [44]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [45]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [46]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER_LAND').getInfo()

    print('Image #',i,date,'Cloud cover:',cloud)

    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2018-10-07 Cloud cover: 0.05


Image # 2 2018-11-08 Cloud cover: 11.84


Image # 5 2018-12-26 Cloud cover: 5.99


## NDVI Analysis

In [47]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [48]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # print some information
    print('Image #',i,date)

    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 0 2018-10-07


Image # 2 2018-11-08


Image # 5 2018-12-26


## Folium

In [56]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
m

In [51]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [57]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']),
                        ndvi_parameters,
                        name=date)

# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [54]:
#save the folium map as an HTML file
my_map.save('camp.html')